<a href="https://colab.research.google.com/github/JoacoPagani/Data-Science-II-Machine-Learning-para-la-Ciencia-de-Datos/blob/main/EntregaFinalP2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

¡¡ Debido a una constante caida del Entorno colab dado que la RAM del sistema se saturaba se tuvo que tomar la medida de separar la entrega en 2 archivos .ipynb, donde el ultimo codigo del anterior es la descarga del Df el cual se sigue trabajando aca !!

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
df = pd.read_csv('df.csv')

#Previo al Entrenamiento

####Separacion de la variable

Debido a que queremos predecir la variable "Price" dividiremos el DataSet en las variables dependientes y independientes

In [3]:
Y_df = df[['price']].copy()
X_df = df.drop('price', axis=1).copy()

Por razones que exceden fue necesario tomar unicamente una parte de los datos, debido a que no era posible entrenar a los modelos

In [4]:
X_df_sampled = X_df.sample(frac=0.3, random_state=42)
Y_df_sampled = Y_df.loc[X_df_sampled.index]

####Eliminacion de variables correlativas

Durante el EDA se realizo la matriz de coeficientes de Correlación de Pearson se encontro una correlacion moderada-fuerte entre las variables "surface_total" y "surface_covered". Por lo que se decide eliminar una de las dos para mejorar la predicción.
Ademas se eliminan las variables "price_period" y "operation_type" ya que todas las filas contaban con el mismo valor. Y la variable "title" que no tiene aporte en el entrenamiento del modelo

In [5]:
X_df_sampled.drop(['surface_covered', 'price_period', 'title', 'operation_type'], axis=1, inplace=True)

####Estandarizacion de variables

Estandarizamos las variables numericas ya que ayuda a mejorar la estabilidad numérica, la convergencia del modelo y la interpretación de los resultados

In [6]:
from sklearn.preprocessing import StandardScaler
# Identificar las columnas numéricas
numerical_cols = X_df_sampled.select_dtypes(include=['number']).columns

# Excluir las columnas de longitud y latitud de la lista de columnas numéricas
numerical_cols = [col for col in numerical_cols if col not in ['lat', 'lon']]

# Reemplazar valores infinitos por NaN
X_df_sampled[numerical_cols] = X_df_sampled[numerical_cols].replace([np.inf, -np.inf], np.nan)

# Rellenar NaN con la media de cada columna
for col in numerical_cols:
    X_df_sampled[col] = X_df_sampled[col].fillna(X_df_sampled[col].mean())

# Instanciar el objeto scaler
scaler = StandardScaler()

# Aplicar la transformación solo a las columnas seleccionadas
X_df_sampled[numerical_cols] = scaler.fit_transform(X_df_sampled[numerical_cols])


Para poder aplicar los modelos de Machine Learning, transformamos las variables categóricas aplicando la codificación One Hot Encoding.

In [7]:
X_df_sampled = pd.get_dummies(X_df_sampled, columns=X_df_sampled.select_dtypes(include=['object']).columns, drop_first=True)

X_df_sampled = X_df_sampled.astype(float)

X_df_sampled

,lat,lon,rooms,bedrooms,bathrooms,surface_total,price_m2,created_on_2018-12-30,created_on_2018-12-31,created_on_2019-01-01,...,l3_Ángel Gallardo,property_type_Casa de campo,property_type_Cochera,property_type_Departamento,property_type_Depósito,property_type_Local comercial,property_type_Lote,property_type_Oficina,property_type_Otro,property_type_PH
49012,-34.624980,-58.451783,-0.044607,-0.058478,-0.776444,-0.095222,1.542547e-01,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
192065,-34.593887,-58.492172,0.003481,-0.058478,2.432097,-0.080619,-1.497962e-01,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
128392,-32.774997,-60.783177,0.003481,-0.058478,-0.000223,-0.040334,-3.775965e-01,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
240900,-34.807937,-58.171665,0.003481,-0.058478,-0.000223,-0.086914,5.100572e-17,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
205239,-34.731537,-58.306966,-0.044607,-0.058478,-0.776444,-0.080116,-3.348535e-01,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
218516,-31.424843,-64.181799,-0.044607,-0.058478,0.293070,-0.098244,1.866920e-01,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
148095,-34.597390,-58.408955,1.424115,1.046725,1.362583,-0.068785,3.536080e-01,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
54234,-34.593887,-58.492172,-1.513329,-0.058478,0.293070,-0.098999,3.464462e-01,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
189619,-34.607756,-58.444935,-0.044607,-0.058478,-0.776444,-0.099251,5.100572e-17,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


#Construccion de Modelos

LLevaremos a cabo los entrenamientos y puesta en prueba del rendimiento de diferentes tipos de modelos.

Dividimos los datos como primer paso, definiendo el conjunto de entrenamiento


In [8]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score


X_train, X_test, y_train, y_test = train_test_split(X_df_sampled, Y_df_sampled, test_size=0.2, random_state=123)

#Veamos como quedaron divididos los conjuntos
X_train.shape, X_test.shape
"""
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)
y_predRF = model.predict(X_test)"""

'\nmodel = RandomForestRegressor(n_estimators=100, random_state=42)\nmodel.fit(X_train, y_train)\ny_predRF = model.predict(X_test)'

In [9]:
# Regression models
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor

from sklearn.model_selection import RandomizedSearchCV

In [10]:
class BaseModel:
    """Clase base con funcionalidad común para modelos de clasificación y regresión"""
    def __init__(self, X_train, y_train, X_test, y_test):
        self.X_train = X_train
        self.y_train = y_train
        self.X_test = X_test
        self.y_test = y_test
        self.models = {}
        self.best_model = None
        self.model_scores = {}

    def save_model(self, filename):
        """Guarda un modelo en disco"""
        if self.best_model:
            with open(filename, 'wb') as f:
                pickle.dump(self.best_model, f)
            print(f"Modelo guardado como {filename}")

    @staticmethod
    def load_model(filename):
        """Carga un modelo desde disco"""
        with open(filename, 'rb') as f:
            return pickle.load(f)
    def save_best_model(self, filepath, save_scaler=True, model_info=True):
        """
        Guarda el mejor modelo junto con información relevante y el scaler si existe.

        Parameters:
        -----------
        filepath : str
            Ruta base donde se guardará el modelo (sin extensión)
        save_scaler : bool, default=True
            Si es True, guarda el scaler junto con el modelo (si existe)
        model_info : bool, default=True
            Si es True, guarda un archivo JSON con información sobre el modelo

        Returns:
        --------
        dict
            Diccionario con las rutas de los archivos guardados
        """
        import os
        import joblib
        import json
        from datetime import datetime

        saved_files = {}

        try:
            # Crear el directorio si no existe
            os.makedirs(os.path.dirname(filepath), exist_ok=True)

            # Guardar el modelo
            model_path = f"{filepath}_model.joblib"
            joblib.dump(self.best_model, model_path)
            saved_files['model'] = model_path

            # Guardar el scaler si existe y se solicita
            if save_scaler and hasattr(self, 'scaler'):
                scaler_path = f"{filepath}_scaler.joblib"
                joblib.dump(self.scaler, scaler_path)
                saved_files['scaler'] = scaler_path

            # Guardar información del modelo
            if model_info:
                info = {
                    'model_type': type(self.best_model).__name__,
                    'timestamp': datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
                    'feature_names': list(self.X_train.columns) if hasattr(self.X_train, 'columns') else None,
                    'parameters': self.best_model.get_params(),
                }

                # Agregar métricas específicas según el tipo de modelo
                if isinstance(self, RegressionModel):
                    best_scores = min(self.model_scores.items(), key=lambda x: x[1]['mse'])
                    info['metrics'] = {
                        'mse': best_scores[1]['mse'],
                        'mae': best_scores[1]['mae'],
                        'r2': best_scores[1]['r2']
                    }
                else:  # ClassificationModel
                    best_scores = max(self.model_scores.items(), key=lambda x: x[1]['accuracy'])
                    info['metrics'] = {
                        'accuracy': best_scores[1]['accuracy'],
                        'classification_report': best_scores[1]['classification_report']
                    }

                info_path = f"{filepath}_info.json"
                with open(info_path, 'w', encoding='utf-8') as f:
                    json.dump(info, f, indent=4)
                saved_files['info'] = info_path

            print(f"Modelo guardado exitosamente en: {model_path}")
            return saved_files

        except Exception as e:
            print(f"Error al guardar el modelo: {str(e)}")
            return None

In [11]:
class RegressionModel(BaseModel):
    def __init__(self, X_train, y_train, X_test, y_test):
        super().__init__(X_train, y_train, X_test, y_test)

    def train_models(self, custom_models=None):
        """Entrena múltiples modelos de regresión con búsqueda de hiperparámetros"""
        default_regressors = {
            'RandomForest': (RandomForestRegressor(), {
                'n_estimators': [50, 100, 200],
                'max_depth': [10, 20, 30],
                'min_samples_split': [2, 5, 10]
            }),
            'LinearRegression': (LinearRegression(), {}),
            'Ridge': (Ridge(), {
                'alpha': [0.1, 1.0, 10.0],
                'solver': ['auto', 'svd', 'cholesky']
            }),
            'Lasso': (Lasso(), {
                'alpha': [0.1, 1.0, 10.0],
                'selection': ['cyclic', 'random']
            }),
            'ElasticNet': (ElasticNet(), {
                'alpha': [0.1, 1.0, 10.0],
                'l1_ratio': [0.2, 0.5, 0.8]
            }),
            'SVR': (SVR(), {
                'C': [0.1, 1, 10],
                'kernel': ['rbf', 'linear'],
                'gamma': ['scale', 'auto']
            }),
            'GradientBoosting': (GradientBoostingRegressor(), {
                'n_estimators': [50, 100, 200],
                'learning_rate': [0.01, 0.1, 0.3],
                'max_depth': [3, 5, 7]
            }),
            'KNN': (KNeighborsRegressor(), {
                'n_neighbors': [3, 5, 7, 9],
                'weights': ['uniform', 'distance'],
                'metric': ['euclidean', 'manhattan']
            })
        }

        # Permite agregar modelos personalizados
        if custom_models:
            default_regressors.update(custom_models)

        for name, (reg, param_grid) in default_regressors.items():
            try:
                print(f"\nEntrenando {name}...")
                search = RandomizedSearchCV(
                    reg,
                    param_grid,
                    n_iter=10,
                    cv=5,
                    scoring='neg_mean_squared_error',
                    n_jobs=-1
                )
                search.fit(self.X_train, self.y_train)
                self.models[name] = search.best_estimator_
                print(f"{name} - Mejores parámetros: {search.best_params_}")
                print(f"{name} - Mejor score CV: {-search.best_score_:.4f}")
            except Exception as e:
                print(f"Error entrenando {name}: {str(e)}")

    def evaluate_models(self):
        """Evalúa todos los modelos entrenados y guarda sus métricas"""
        best_score = float('inf')
        evaluation_results = []

        for name, model in self.models.items():
            try:
                y_pred = model.predict(self.X_test)
                mse = mean_squared_error(self.y_test, y_pred)
                mae = mean_absolute_error(self.y_test, y_pred)
                r2 = r2_score(self.y_test, y_pred)

                self.model_scores[name] = {
                    'mse': mse,
                    'mae': mae,
                    'r2': r2
                }

                evaluation_results.append({
                    'Model': name,
                    'MSE': mse,
                    'MAE': mae,
                    'R2': r2
                })

                if mse < best_score:
                    self.best_model = model
                    best_score = mse

                print(f"\nResultados para {name}:")
                print(f"MSE: {mse:.4f}")
                print(f"MAE: {mae:.4f}")
                print(f"R2: {r2:.4f}")

            except Exception as e:
                print(f"Error evaluando {name}: {str(e)}")

        # Crear DataFrame con resultados
        results_df = pd.DataFrame(evaluation_results)
        print("\nResumen de resultados:")
        print(results_df.sort_values('MSE'))
        print(f"\nMejor modelo: {type(self.best_model).__name__} con MSE: {best_score:.4f}")
        return results_df

In [12]:
# Crear instancia
model = RegressionModel(X_train, y_train, X_test, y_test)

# Entrenar modelos con búsqueda de hiperparámetros
model.train_models()

# Evaluar modelos en el conjunto de prueba
results_df = model.evaluate_models()


Entrenando RandomForest...


KeyboardInterrupt: 

In [ ]:
from sklearn.model_selection import cross_val_score

# Evaluación con validación cruzada (usando MAE como métrica)
mae_cv_scores = cross_val_score(best_model_gb, X_train, y_train, cv=5, scoring='neg_mean_absolute_error')
print(f"Cross-validated MAE: {-mae_cv_scores.mean():.2f}")


In [ ]:
# Predicción en el conjunto de prueba
y_predGB = best_model_gb.predict(X_test)

# Calcular métricas
mae = mean_absolute_error(y_test, y_predGB)
mse = mean_squared_error(y_test, y_predGB)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_predGB)

print(f"Mean Absolute Error (MAE): {mae}")
print(f"Mean Squared Error (MSE): {mse}")
print(f"Root Mean Squared Error (RMSE): {rmse}")
print(f"R-squared (R2): {r2}")
